In [1]:
import matplotlib.pyplot as plt
import numpy as np

%run functions/preprocess.py
%run functions/linear_regression.py

np.set_printoptions(formatter={'float_kind':'{:f}'.format})
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.style.use('seaborn')

## Loading the data

In [ ]:
path = open('path.txt', 'r').read().strip()
df_data = pd.read_pickle(path)
df_data = df_data.drop(['time of day', 'weekday', 'Unix', 'next_event', 'prev_event', 
                        'nextUnix', 'prevUnix', 'prevTime'], axis=1)

In [ ]:
np.random.seed(42)

df_data = df_data[df_data['case concept:name'].isin(np.random.choice(df_data['case concept:name'].unique(), size=7000, replace=False))]

In [ ]:
from sklearn.model_selection import train_test_split

df_data, df_test = train_test_split(df_data, test_size=0.2, shuffle=False)

In [ ]:
def encode(df):
    """One hot encodes columns"""
    df = one_hot_encode(df, 'event Action', 'action')
    df = one_hot_encode(df, 'case LoanGoal', 'loangoal')
    df = one_hot_encode(df, 'case ApplicationType', 'appl_type')
    df = one_hot_encode(df, 'event org:resource', '')
    df = one_hot_encode(df, 'event concept:name', 'eventname')
    df = one_hot_encode(df, 'event EventOrigin', 'origin')
    df = one_hot_encode(df, 'event lifecycle:transition', 'lifecycle')
    return df
    
df_data = encode(df_data)

In [ ]:
def time_diff(df):
    """Calculates time difference between i and i+1 
    within a trace and converts the value to log"""
    df['time_diff'] = (df['nextTime'] - df['event time:timestamp']).dt.total_seconds()
    df = df.dropna().reset_index(drop=True)
    
    df = df[df['time_diff']>=0]
    df['time_diff'] = np.log(df['time_diff'].replace(0, np.nan))
    df['time_diff'] = df['time_diff'].replace(np.nan, 0)
    
    return df.drop(['event time:timestamp', 'nextTime'], axis=1)

df_data = time_diff(df_data)

In [ ]:
df_data.groupby('case concept:name').apply(lambda x: sliding_window(3, x))

""


In [ ]:
test = cross_validate(np.array(X), np.array(Y))
model = test[1]

---

## Testing

In [ ]:
encode(df_test)
time_diff(df_test)


0

In [ ]:
model = test[1]
test = test[0]
mse = np.mean([mean_squared_error(i[0], i[1]) for i in test])
r2 = np.mean([r2_score(i[0], i[1]) for i in test])
mae = np.mean([mean_absolute_error(i[0], i[1]) for i in test])
rmse = np.mean([mean_squared_error(i[0], i[1], squared=False) for i in test])
# print(f'mse: {math.exp(mse)}\nr^2: {r2}\nrmse: {math.exp(rmse)}\nmae:{math.exp(mae)}')
print(f'mse: {mse}\nr^2: {r2}\nrmse: {rmse}\nmae:{mae}')

NameError: name 'test' is not defined

In [ ]:
# cols = df_data.columns.tolist()
# cols.remove('time_diff')
# cols = cols + ['time_diff']
# df_data = df_data[cols]

# categorial = ['case LoanGoal']
# numerical = ['case RequestedAmount']
# df_data.drop(categorial, axis=1).drop(numerical, axis=1)
